In [1]:
import pandas as pd
import geopandas as gpd
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
from statsmodels.tools.tools import add_constant
import numpy as np
import statsmodels.api as sm

/Users/Salim/Library/Python/3.8/lib/python/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


### Load data

Load dataset with snapshot of 2014 pipeline snapshot and outcomes.

In [2]:
mv_history = pd.read_csv('./data/pipeline_history.csv')

The variable 'Built' indicates how much of the project was built by June 2022. The variable 'Units' indicates how large the project was as of June 2014 - or, if the project had no size listed in 2014, then the eventual size of the project. The variable 'Pg' indicates what page of the [2014 Planning Division Update](https://web.archive.org/web/20140709150825/http://mountainview.gov/civicax/filebank/blobdload.aspx?BlobID=13170) the project is listed on. This dataframe was created manually by reading the 2014 PDU.

In [3]:
mv_history.head()

,address,page,pct_built,units
0,420 San Antonio Road,3,1.00,373
1,2580 California Street,4,0.26,632
2,1701 W. El Camino Real,7,1.00,24
3,1101 W. El Camino Real,8,1.00,52
4,801 W. El Camino Real,8,1.00,164


Load site inventory datasets.

In [4]:
si_geo = gpd.read_file('./data/MV_Site_Inventory/MV_Site_Inventory.shp')

In [5]:
si = pd.read_csv('./data/hcd_table_a.csv', low_memory=False)
si = si[~si.isna().all(axis=1)]
si = si[:-1]

In [6]:
permits = gpd.read_file('./data/all_permits.json')

In [7]:
zoning = gpd.read_file('./data/Zoning_Districts/Zoning_Districts.shp')

### Pending Projects History Analysis

In [9]:
sum(mv_history.units * mv_history.pct_built) / mv_history.units.sum()

0.6646353322528363

What would a 33% discount factor do to the city's projections?

In [47]:
# city projections
tot_pipe = 6913
li_pipe = 1896

# expected total units
sum(mv_history.units * mv_history.pct_built) / mv_history.units.sum() * tot_pipe

4594.624051863858

In [48]:
# delta
tot_pipe - sum(mv_history.units * mv_history.pct_built) / mv_history.units.sum() * tot_pipe

2318.375948136142

In [49]:
# expected li units 
sum(mv_history.units * mv_history.pct_built) / mv_history.units.sum() * li_pipe

1260.1485899513777

In [50]:
# delta (li)
li_pipe - sum(mv_history.units * mv_history.pct_built) / mv_history.units.sum() * li_pipe

635.8514100486223

In [46]:
pearsonr(mv_history.units, mv_history.pct_built)

(-0.15362040968120402, 0.5178707812714507)

### Pending Projects Predictions

In [66]:
np.random.seed(0)
reg = sm.Logit(mv_history.pct_built, add_constant(mv_history.units)).fit()

def predict_success(n_units):
    """P(devs) adjusted for number of units in project."""
    return reg.predict([1, n_units])

Optimization terminated successfully.
         Current function value: 0.536679
         Iterations 5


In [67]:
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              pct_built   No. Observations:                   20
Model:                          Logit   Df Residuals:                       18
Method:                           MLE   Df Model:                            1
Date:                Wed, 13 Jul 2022   Pseudo R-squ.:                0.008282
Time:                        12:01:41   Log-Likelihood:                -10.734
converged:                       True   LL-Null:                       -10.823
Covariance Type:            nonrobust   LLR p-value:                    0.6720
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.3975      0.655      2.134      0.033       0.114       2.681
units         -0.0017      0.003     -0.656      0.512      -0.007       0.003
==============================================================================
"""

In [52]:
def predict_success_floor(n_units):
    """
    P(devs) adjusted for number of units in project, with a floor that 
    pipeline sites are at least as likely to be developed as opportunity sites.
    """
    return max(reg.predict([1, n_units]), .206)

In [53]:
pipeline = si[si['Optional Information 1'].str.contains('Pending')]

This dataset excludes approved projects, which is why it has less total capacity than the draft claims.

In [54]:
pipeline['Total Capacity'].sum()

6913.0

In [55]:
pipeline['Site Status'].value_counts()

Pending Project    66
Name: Site Status, dtype: int64

#### Pipeline capacity with plain Logistic Regression

In [56]:
p_devs = pipeline['Total Capacity'].apply(predict_success)

In [57]:
sum(p_devs * pipeline['Total Capacity']).item()

2231.4679642693827

In [58]:
sum(p_devs * pipeline['Lower Income Capacity']).item()

920.160886663414

#### Pipeline capacity with constraint that P(dev | pipeline) > P(dev | opp) for all sites

In [59]:
p_devs = pipeline['Total Capacity'].apply(predict_success_floor)

In [60]:
sum(p_devs * pipeline['Total Capacity']).item() 

2872.4262057831024

In [61]:
tot_pipe - sum(p_devs * pipeline['Total Capacity']).item() 

4040.5737942168976

In [62]:
sum(p_devs * pipeline['Lower Income Capacity']).item()

1020.7332348117777

In [63]:
li_pipe - sum(p_devs * pipeline['Lower Income Capacity']).item() 

875.2667651882223

### Opportunity Sites

In [ ]:
opps = si[~si['Site Status'].str.contains('Pending')]

In [ ]:
opps['Zoning Designation (Current)'].value_counts()

Find opportunity sites for each of four major precise plan areas.

In [ ]:
ecr = opps[opps['Zoning Designation (Current)'].str.contains("El Camino Real")]

In [ ]:
ew = opps[opps['Zoning Designation (Current)'].str.contains("East Whisman")]

In [ ]:
sa = opps[opps['Zoning Designation (Current)'].str.contains("San Antonio")]

In [ ]:
nb = opps[opps['Zoning Designation (Current)'].str.contains("North Bayshore")]

There are 55 opportunity sites in El Camino Real Precise Plan, 15 in the East Whisman precise plan, 9 in the San Antonio precise plan, and 6 in the North Bayshore precise plan.

In [ ]:
len(ecr), len(ew), len(sa), len(nb)

### Cleaning permits dataset

Half of older permits have same geometry.

In [ ]:
permits.apn = permits.apn.str.split('-').str.join('')

In [ ]:
permits.apn = permits.apn.str.replace('Â\xa0', '').values.tolist()

In [ ]:
permits.apn = permits.apn.str.strip()

In [ ]:
permits = permits[(~permits.apn.duplicated()) | (permits.apn.isnull())]

In [ ]:
permits = gpd.sjoin(permits, zoning.to_crs('EPSG:4326'))

In [ ]:
plt.hist(permits[permits.PRECPLAN == 'P(39)'].permyear)

In [ ]:
pp_permits = permits['PRECPLAN'].value_counts()

In [ ]:
ecr_ppid = 'P(38)'
nbs_ppid = 'P(39)'
sa_ppid = 'P(40)'
ew_ppid = 'P(41)'

In [ ]:
ecr_yrs, ew_yrs, sa_yrs, nbs_yrs = 8, 3, 8, 5

In [ ]:
ecr_exp = pp_permits[ecr_ppid] / ecr_yrs * 8

In [ ]:
pp_permits[ecr_ppid] / ecr_yrs

In [ ]:
nbs_exp = pp_permits[nbs_ppid] / nbs_yrs * 8

In [ ]:
pp_permits[nbs_ppid] / nbs_yrs

In [ ]:
sa_exp = pp_permits[sa_ppid] / sa_yrs * 8

In [ ]:
pp_permits[sa_ppid] / sa_yrs

In [ ]:
ew_exp = pp_permits[ew_ppid] / ew_yrs * 8

In [ ]:
pp_permits[ew_ppid] / ew_yrs

In [ ]:
for pp, name, expect in zip([ecr, ew, sa, nb], ['ecr', 'ew', 'sa', 'nb'], [ecr_exp, ew_exp, sa_exp, nbs_exp]):
    print('For', name, 'the city claims', len(pp), 'projects in 8 years', ' but historical trends suggest')
    print(int(round(expect, 0)), "is more reasonable. That'd discount their site capacity claims by", round(1 - (int(round(expect,0)) / len(pp)), 2), '%')
    print('\n')

In [ ]:
for pp, name, expect in zip([ecr, ew, sa, nb], ['ecr', 'ew', 'sa', 'nb'], [ecr_exp, ew_exp, sa_exp, nbs_exp]):
    print('For', name, 'the city claims', len(pp), 'projects in 8 years, but historical trends suggest')
    print(int(round(expect, 0)), "is more reasonable. That'd inflate their site capacity claims by", 
          int(round(round(len(pp) / expect, 3)*100 - 100, 0)), '%')
    print('\n')

In [ ]:
ecr_li, ecr_tot = 1283, 2530
ew_li, ew_tot = 997, 1312
sa_li, sa_tot = 182, 325
nb_li, nb_tot = 313, 405
run_sum_li, run_sum_tot = 0, 0

In [ ]:
for pp, name, expect, li, tot in zip([ecr, ew, sa, nb], 
                                ['ecr', 'ew', 'sa', 'nb'], 
                                [ecr_exp, ew_exp, sa_exp, nbs_exp],
                                [ecr_li, ew_li, sa_li, nb_li],
                                [ecr_tot, ew_tot, sa_tot, nb_tot]):
    print('For', name, li*(1 - expect / len(pp)))
    print('\n')
    print('For', name, tot*(1 - expect / len(pp)))
    print('\n')
    run_sum_li += li*(1 - expect / len(pp))
    run_sum_tot += tot*(1 - expect / len(pp))

In [ ]:
run_sum_li

In [ ]:
run_sum_tot

In [ ]:
5502 - run_sum_tot

In [ ]:
3240 - run_sum_li

In [ ]:
2775 + 465


In [ ]:
4698 + 804